In [0]:
/*
================================================================================
DDL Script: Create Gold Views
================================================================================
Script Purpose: 
  This script creates views for the Gold Layer in the data warehouse.
  The Gold layer represents the final dimension and fact tables (start schema)
  
  Each view performs transformations and combines data from the Silver layer to
  produce a clean, enriched, and business-ready dataset.

Usage: 
  - These views can be queried directly for analytics and reporting.
================================================================================
*/

-- ===================================
-- Create Dimension: gold.dim_customer
-- ===================================
CREATE OR REPLACE VIEW datawarehouse.gold.dim_customer 
AS 
select 
  row_number() over (order by ci.cst_id) as customer_key,
  ci.cst_id as customer_id,
  ci.cst_key as customer_number,
  ci.cst_firstname as first_name,
  ci.cst_lastname as last_name,
  la.CNTRY as country,
  ci.cst_marital_status as marital_status,
  case
    when ci.cst_gndr != 'n/a' then ci.cst_gndr -- CRM is the Master table to choose first
    else coalesce(ca.GEN, 'n/a')
  end as gender,
  ca.BDATE as birthdate,
  ci.cst_create_date as create_date
from datawarehouse.silver.crm_cust_info ci
left join datawarehouse.silver.erp_cust_az12 ca
  on ci.cst_key = ca.cid
left join datawarehouse.silver.erp_loc_a101 la
  on ci.cst_key = la.CID
order by cst_firstname;




-- ===================================
-- Create Dimension: gold.dim_product
-- ===================================
CREATE OR REPLACE VIEW datawarehouse.gold.dim_prodcut
AS 
select 
row_number() over (order by p.prd_start_dt, p.prd_key) as product_key,
p.prd_id as product_id,
p.prd_key as product_number,
p.prd_nm as prodcut_name,
p.cat_id as category_id,
ca.cat as category,
ca.subcat as subcategory,
ca.maintenance,
p.prd_line as product_line,
p.prd_cost as cost,
p.prd_start_dt as start_date
-- p.prd_end_dt -- Always be NULL cuz of the wher condition 

from datawarehouse.silver.crm_prd_info p
left join datawarehouse.silver.erp_px_cat_g1v2 ca
  on ca.id = p.cat_id
where p.prd_end_dt is null; -- Filter out all historical data;




-- ===================================
-- Create Fact: gold.fact_sales
-- ===================================
CREATE OR REPLACE VIEW datawarehouse.gold.fact_sales 
AS 
select 
sd.sls_ord_num as order_number,
pr.product_key,
cu.customer_key,
sd.sls_order_dt as order_date,
sd.sls_ship_dt as ship_date,
sd.sls_due_dt as due_date,
sd.sls_sales as sales_amount,
sd.sls_quantity as quantity,
sd.sls_price as price
from datawarehouse.silver.crm_sales_details sd
left join datawarehouse.gold.dim_prodcut pr
  on sd.sls_prd_key = pr.product_number
left join datawarehouse.gold.dim_customer cu
  on sd.sls_cust_id = cu.customer_id;


select * from datawarehouse.gold.fact_sales;